In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 

import tensorflow as tf
from keras import utils as np_utils 

from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout, GlobalAveragePooling1D
from keras.layers import Input
from keras.models import Model
from keras.layers import BatchNormalization
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import MaxPooling1D

import pickle


In [ ]:
from sklearn.preprocessing import StandardScaler
df = pd.read_pickle("/kaggle/input/asrqw334/data_final_s2_300s_1000hz_RT1purie.pkl") # index column을 0번째 줄로 정한다는 의미입니다. 

df['label'].value_counts()
df_y=df['label']
df_x=df.drop(['label'],axis=1)

# StandardScaler 객체 생성
scaler = StandardScaler()

# 데이터에 스케일링 적용
scaled_ecg_data = scaler.fit_transform(df_x)

# 스케일링된 데이터를 DataFrame 형태로 변환
scaled_ecg_data_df = pd.DataFrame(scaled_ecg_data, columns=df_x.columns)

# 스케일링된 데이터의 첫 5행을 출력
df = pd.concat([scaled_ecg_data_df, df_y], axis = 1) # 열 방향으로 합치기
df

In [ ]:
# train_test를 분리합니다. 
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size = 0.1, stratify = df['label'])
print(train_df.shape, valid_df.shape)

In [ ]:
train_df['label'].value_counts()

In [ ]:
batch_size = 8
epochs = 100

## 조심 
patience = 30 # earlystop 할 때 조절하는 것 
## 고정
data_len = len(df.iloc[0]) - 1

In [ ]:
# dataset, dataloader

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size = 32, train = False):
        self.df = df.reset_index(drop=True)
        self.train = train
        self.batch_size = batch_size
    
    def __len__(self):
        return np.ceil(len(self.df) / self.batch_size).astype(int)
    
    def on_epoch_end(self):
        if self.train: # Reshuffle train on end of epoch
            self.df = self.df.sample(frac=1.0).reset_index(drop=True)
            
    def __getitem__(self, idx):
        batch_x = self.df.iloc[idx*self.batch_size:(idx+1)*self.batch_size, :-1].values # batch_size, 30000
        batch_y = self.df.iloc[idx*self.batch_size:(idx+1)*self.batch_size].label.values # batch_size, 1

        X = np.zeros((batch_x.shape[0], data_len))
        
        # train -> augmentation O
        if self.train:
          for i in range(batch_x.shape[0]): # i = 0, 1, 2, ,,, , 31
            X[i] = batch_x[i] + np.random.normal(0, 0.05, size = (data_len,)) 
        
        # valid -> augmentation X
        else:
          X = batch_x

        # X의 차원을 늘려줍니다. <- model 들어갈 때 error 뜸
        X = tf.expand_dims(X,2)

        return X, batch_y 

In [ ]:
train_ds = DataGenerator(train_df, batch_size = batch_size, train = False)
valid_ds = DataGenerator(valid_df, batch_size = batch_size, train = False)

for x_train, y_train in train_ds:
  print(x_train.shape)
  print(y_train.shape)
  break
y_train

In [ ]:
def network_v3():

    # 마지막 차원 추가해줌 위에서
    x = Input(shape = (data_len, 1), name='inputs_cnn')

    # Conv1D
    y = Convolution1D(16, 55, activation = 'leaky_relu')(x)
    y = BatchNormalization()(y)
    y = MaxPool1D(10)(y)
    y = Dropout(0.2)(y)
    
    y = Convolution1D(32, 55, activation = 'leaky_relu')(x)
    y = BatchNormalization()(y)
    y = MaxPool1D(10)(y)
    y = Dropout(0.2)(y)

    y = Convolution1D(64, 25, activation = 'leaky_relu')(y)
    y = BatchNormalization()(y)
    y = MaxPool1D(5)(y)
    y = Dropout(0.2)(y)

    y = Convolution1D(128, 10, activation = 'leaky_relu')(y)
    y = BatchNormalization()(y)
    y = MaxPool1D(5)(y)
    y = Dropout(0.2)(y)

    y = Convolution1D(256, 10, activation = 'leaky_relu')(y)
    y = BatchNormalization()(y)
    y = MaxPool1D(5)(y)
    y = Dropout(0.2)(y)

    y = Convolution1D(512, 9, activation = 'leaky_relu')(y)

    # GAP
    y = GlobalAveragePooling1D()(y)

    # Dense
    y = Dense(32, activation='leaky_relu')(y)
    
    y = Dense(1, activation='sigmoid', name='main_output')(y) 

    # Model
    model = Model(x, y)

    # return
    return model

model = network_v3()
model.summary()

In [ ]:
# compile

loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

model.compile(loss = loss, optimizer = optimizer, metrics = ['acc'])

es = EarlyStopping(monitor = 'val_loss', patience = patience, restore_best_weights=True)

In [ ]:
# history_v1 = model.fit(train_ds, epochs = epochs, validation_data = valid_ds, callbacks = [es])
history_v1 = model.fit(train_ds, epochs = epochs, validation_data = valid_ds)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def show_lcurve(hists, titles, colors=["r","g","b","k"], size=(12,4), x_itv=1, lw=1):
    plt.figure(figsize=size)
    plt.style.use("seaborn-white")
    # loss 
    plt.subplot(121)
    for i, hist in enumerate(hists):
        loss = hist.history['loss']
        val_loss = hist.history['val_loss']
        epochs = range(1, 1+len(loss))
        xbins = range(0, len(loss)+1, x_itv)
        plt.plot(epochs, loss, lw=lw,linestyle=':', label=f'{titles[i]} train loss', c=colors[i])
        plt.plot(epochs, val_loss, lw=lw,marker='.', label=f'{titles[i]} valid loss', c=colors[i])
        plt.legend();plt.grid(True);plt.xticks(xbins)
        plt.xlabel('Epochs');plt.ylabel('Loss')
        x, y = epochs[-1], hist.history['loss'][-1]
        plt.text(x, y, np.round(y,2), c=colors[i])
        x, y = epochs[-1], hist.history['val_loss'][-1]
        plt.text(x, y, np.round(y,2), c=colors[i])

    # acc
    plt.subplot(122)
    for i, hist in enumerate(hists):
        acc = hist.history['acc']
        val_acc = hist.history['val_acc']
        plt.plot(epochs, acc, linestyle=':', lw=lw,label=f'{titles[i]} train acc', c=colors[i])
        plt.plot(epochs, val_acc, marker='.', lw=lw,label=f'{titles[i]} valid acc', c=colors[i])
        plt.legend();plt.grid(True);plt.xticks(xbins)
        plt.xlabel('Epochs');plt.ylabel('Acc')
        x, y = epochs[-1], hist.history['acc'][-1]
        plt.text(x, y, np.round(y,2), c=colors[i])
        x, y = epochs[-1], hist.history['val_acc'][-1]
        plt.text(x, y, np.round(y,2), c=colors[i])
    plt.show()

In [ ]:
show_lcurve([history_v1], ["hist"], size = (22, 5), x_itv = 10 ) # x_itv: epoch 몇번씩 볼 수 있는지